# ProyectoDSParteI_Hamburg

## Abstracto

Este proyecto explora relaciones entre variables fisicoquímicas del vino tinto y la variable de **calidad** registrada por catadores en una escala ordinal. El objetivo es construir una primera narrativa descriptiva, establecer preguntas e hipótesis y responderlas con resúmenes numéricos y visualizaciones univariadas, bivariadas y multivariadas, siguiendo las prácticas vistas: manejo de datos con **Pandas**, visualización con **Matplotlib/Seaborn** y estadística descriptiva básica.

Trabajamos con el archivo **`winequality-red.csv`** (muestras de vinos tintos portugueses). Las columnas incluyen acidez fija, acidez volátil, ácido cítrico, azúcar residual, cloruros, dióxido de azufre libre y total, densidad, pH, sulfatos, alcohol y una calidad global. Las preguntas guía son: ¿qué variables muestran mayor asociación con la **calidad**?, ¿existen patrones que sugieran mejoras en la producción?, ¿qué tan frecuentes son outliers o datos ausentes?

Hipótesis de interés: (H1) **alcohol** se asocia positivamente con la calidad; (H2) **acidez volátil** se asocia negativamente con la calidad; (H3) **sulfatos** y **SO₂ total** pueden mejorar la calidad por su efecto antioxidante; (H4) **densidad** estará fuertemente relacionada con **azúcar residual**. Para validarlas, construiremos histogramas, boxplots, diagramas de dispersión con codificación por color/tamaño y un mapa de correlaciones. También cuantificamos valores ausentes y outliers mediante IQR.

Los hallazgos preliminares indican que la **calidad** varía principalmente con **alcohol** (positiva) y **acidez volátil** (negativa), mientras que azúcar residual y densidad describen una relación fuerte entre sí, más estructural que sensorial. No se observan datos ausentes y la distribución de calidades es desequilibrada hacia valores 5–6. Estos resultados fundamentan líneas futuras: modelado supervisado simple (p. ej., regresión logística o árboles) y un análisis más fino de interacciones, manteniendo el foco en interpretabilidad y buenas prácticas visuales.


## Preguntas e hipótesis

**P1.** ¿Mayor **alcohol** implica mayor **calidad**?  
**H1.** La relación es positiva.

**P2.** ¿La **acidez volátil** degrada la **calidad**?  
**H2.** La relación es negativa.

**P3.** ¿**sulfatos** y **SO₂ total** se asocian a mejor **calidad**?  
**H3.** Efecto positivo moderado.

**P4.** ¿**densidad** se explica por **azúcar residual**?  
**H4.** Relación positiva alta.


In [ ]:

!wget -q -O winequality-red.csv https://raw.githubusercontent.com/gerardohamburg/CoderHouse/main/winequality-red.csv

import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
df = pd.read_csv("winequality-red.csv", sep=";")
df.columns = (df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("/", "_", regex=False))
df.head()


In [ ]:

df.shape, df.dtypes


In [ ]:

na = df.isna().sum().to_frame("faltantes")
na.T


In [ ]:

desc = df.describe().T
desc


In [ ]:

df["quality"].value_counts().sort_index()


In [ ]:

fig, axs = plt.subplots(2, 2, figsize=(10, 7))
sns.histplot(df["alcohol"], kde=True, ax=axs[0,0])
axs[0,0].set_title("Alcohol")
sns.histplot(df["volatile_acidity"], kde=True, ax=axs[0,1])
axs[0,1].set_title("Acidez volátil")
sns.histplot(df["residual_sugar"], kde=True, ax=axs[1,0])
axs[1,0].set_title("Azúcar residual")
sns.histplot(df["sulphates"], kde=True, ax=axs[1,1])
axs[1,1].set_title("Sulfatos")
plt.tight_layout()
plt.show()


In [ ]:

fig, axs = plt.subplots(1, 3, figsize=(15, 4))
sns.boxplot(data=df, x="quality", y="alcohol", ax=axs[0])
axs[0].set_title("Alcohol vs Calidad")
sns.boxplot(data=df, x="quality", y="volatile_acidity", ax=axs[1])
axs[1].set_title("Acidez volátil vs Calidad")
sns.scatterplot(data=df, x="residual_sugar", y="density", ax=axs[2])
axs[2].set_title("Azúcar residual vs Densidad")
plt.tight_layout()
plt.show()


In [ ]:

plt.figure(figsize=(7,5))
sns.scatterplot(data=df, x="alcohol", y="volatile_acidity", hue="quality", size="sulphates", sizes=(20,120), alpha=0.8)
plt.title("Alcohol vs Acidez volátil con color por Calidad y tamaño por Sulfatos")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0.0)
plt.tight_layout()
plt.show()


In [ ]:

corr = df.corr(numeric_only=True)
plt.figure(figsize=(9,7))
sns.heatmap(corr, cmap="RdBu_r", center=0, annot=False)
plt.title("Mapa de correlaciones")
plt.tight_layout()
plt.show()
corr["quality"].sort_values(ascending=False)


In [ ]:

def iqr_bounds(s):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    low, high = q1 - 1.5*iqr, q3 + 1.5*iqr
    return low, high

for col in ["volatile_acidity", "alcohol"]:
    low, high = iqr_bounds(df[col])
    outs = df[(df[col] < low) | (df[col] > high)][col]
    print(col, "outliers:", outs.shape[0])

fig, axs = plt.subplots(1,2, figsize=(10,4))
sns.boxplot(y=df["volatile_acidity"], ax=axs[0])
axs[0].set_title("Acidez volátil")
sns.boxplot(y=df["alcohol"], ax=axs[1])
axs[1].set_title("Alcohol")
plt.tight_layout()
plt.show()


## Interpretaciones

1. No se detectan valores ausentes. La calidad está concentrada en 5–6.  
2. Los histogramas muestran alcohol con sesgo positivo leve y acidez volátil con cola derecha.  
3. Boxplots por calidad confirman: alcohol creciente con calidad y acidez volátil decreciente.  
4. Azúcar residual y densidad evidencian relación positiva fuerte.  
5. El scatter multivariado sugiere que vinos de mayor calidad combinan mayor alcohol con menor acidez volátil y sulfatos moderados.  
6. El mapa de correlaciones clasifica a alcohol como correlación positiva más alta con calidad, y acidez volátil como la más negativa.  
7. IQR identifica outliers, especialmente en acidez volátil; conviene revisarlos antes de modelar.
